# Program to investigate Scaling with time-steplength dt

In [ ]:
# Plots in "*-CompassRose" FRAMES to investigate numerical scaling
# Code for Chin's 9-Exponential Propagators DOI: 10.1103/PhysRevE.80.037701
#  for non-separable Hamiltonians, here: KERR-oscillator 
# Modification for only 7 Exponentials needs a CorrectionPotential_7Exp(x)
# Expected Energy-Scaling O(dt^2) for SEPARABLE Hamiltonian (STRANG splitting)

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import copy
import scipy.fftpack as fftpack
import scipy.integrate as integrate
import scipy.stats as stats
import time
start_time = time.time()
import WignerPlot_221104

KerrCase = True #False #
# In KerrCase: Use 7 Exponentials (DEFAULT: 9 Exponentials)
exp7 = False #True #
 
if KerrCase: # Kerr (NONseparable Hamiltonian)
    motto=f"Kerr_{'7_Exp' if exp7 else '9_Exp'}"
    dirLabel='KERR'
    def Hamiltonian(x,p):
        return (p**2 + x**2)**2/4
else: # Default: SEPARABLE Hamiltonian
    motto=f"V(x)=0.05*1*x**4"
    dirLabel='V(x4)'
    def Hamiltonian(x,p):
        return Kinetic(p) + Potential(x)
    
# Try to reach 2nd Order using STRANG splitting
StrangSplitting = True # False # 

# Overlap with 2-hump Schroedinger cat state after running for half a period
only_run_until_HalfTime = False #True #
if only_run_until_HalfTime:
    motto=f"HALF_run_{motto}"

In [ ]:
discretizationX = 512//2           # Number of GRID-points in x
discretizationP = discretizationX  # Number of GRID-points in p
                                   # have to be equal, is seems?
# time steps
list_dt_4_separableSystem = [0.05, 0.01]#, 0.001, 0.0005]#0.04, 0.01, 0.005]
#list_dt_4_Kerr = [0.01, 0.005, 0.001]#, 0.0001]#, 0.00002] #Total Calculation Time: [26913.4 sec] @ 512
list_dt_4_Kerr = [0.0015, 0.0005, 0.0001]
#list_dt_4_Kerr = [0.01, 0.002]#, 0.00002]
    
# Offset parameters of Initial (Coherent) State |x0,p0>
M_SQRT1_2 = 1./np.sqrt(2)
R = 3  # Distance from the origin
initial_offsets = [( R*M_SQRT1_2, R*M_SQRT1_2), 
                   ( R*1,         0), 
                   ( R*M_SQRT1_2,-R*M_SQRT1_2), 
                   ( 0,          -R*1), 
                   (-R*M_SQRT1_2,-R*M_SQRT1_2), 
                   (-R*1,         0), 
                   (-R*M_SQRT1_2, R*M_SQRT1_2), 
                   ( 0,           R*1)]

#initial_offsets = [( R*M_SQRT1_2, R*M_SQRT1_2)]#,
#                    ( R*1,         0)]#, 
#                    # ( R*M_SQRT1_2,-R*M_SQRT1_2)]

# '*-CompassRose' FRAMES into which we plot the above
subplot_indices = [(0,2),(1,2),(2,2),(2,1),(2,0),(1,0),(0,0),(0,1)]
# data storage for later plotting of FRAMES
energy_data = [[[],[],[]],[[],[],[]],[[],[],[]]]
WoW_data    = copy.deepcopy(energy_data)
DWL2_data   = copy.deepcopy(energy_data)
DWLoo_data  = copy.deepcopy(energy_data)

# Parameters
mass = 1.            
k = 1.                # Harmonic oscillator spring constant
amplitudeX = 9.0      # xrange = [-amplitudeX,amplitudeX]
amplitudeP = 16.0-7.0 # prange = [-amplitudeP,amplitudeP]
hbar = 1              # Planck constant
L = 1.                # Kerr lambda
period = np.pi/L if KerrCase else 2*np.pi
if only_run_until_HalfTime and KerrCase:
    period = period / 2

In [ ]:
# Define   x, p, theta, lambda VECTORS
xvector = \
    np.linspace(-np.abs(amplitudeX), np.abs(amplitudeX*(1.-2./discretizationX)), discretizationX)
pvector = \
    np.linspace(-np.abs(amplitudeP), np.abs(amplitudeP*(1.-2./discretizationP)), discretizationP)
thetavector = fftpack.fftshift( \
    2.*np.pi*fftpack.fftfreq(pvector.size, pvector[1] - pvector[0]))
lambdavector = fftpack.fftshift( \
    2.*np.pi*fftpack.fftfreq(xvector.size, xvector[1] - xvector[0]))
# Define X, P, Lambda, Theta GRIDS
Theta, X = np.meshgrid(thetavector,  xvector)
P,Lambda = np.meshgrid(pvector, lambdavector)

In [ ]:
# Representation Transformations:

#   W(x,p)            (p <-> theta)      B(x,theta)
#   | (x <-> lambda)                     | (x <-> lambda)
#   A(lambda,p)       (p <-> theta)      Z(lambda,theta)

def X_P__To__X_Theta(W):
    return fftpack.fft(W, overwrite_x=True, axis=1)

def X_Theta__To__X_P(W):
    return fftpack.ifft(W, overwrite_x=True, axis=1)

def X_P__To__Lambda_P(W):
    return fftpack.fft(W, overwrite_x=True, axis=0)

def Lambda_P__To__X_P(W):
    return fftpack.ifft(W, overwrite_x=True, axis=0)

In [ ]:
if StrangSplitting == True:
    halveStep = 0.5
else:
    halveStep = 1.

if exp7 == True:
    keep_if_9Exp_case=0
    def CorrectionPotential_7Exp(x):
        return - x**6 /(9.0 * np.sqrt(2.0) * t2**3)
else:
    keep_if_9Exp_case=1  
    def CorrectionPotential_7Exp(x):
        return 0

In [ ]:
def MoyalGenerator_G(dt, func, mult, r, s, achse, caseSeparable = 'Separable'):
    '''Moyal Superoperator Eq.(10) of DOI: 10.1103/PhysRevA.92.042122  '''
    if caseSeparable == 'Separable':
        eps = dt
    else:
        eps = np.cbrt(dt)
        
    return fftpack.ifftshift( \
        np.exp(-1j*eps*mult*(func(r - hbar* s/2.) - func(r + hbar* s/2.))), axes=(achse,))

In [ ]:
def KineticStep(W,expOp,unusedSlot):
    ''' "Kinetic" Lambda_P - Step '''
    W = X_P__To__Lambda_P(W)
    W *=expOp
    W = Lambda_P__To__X_P(W)
    return W

def PotentialStep(W,expOp,unusedSlot):
    ''' "Potential" X_Theta - Step '''
    W = X_P__To__X_Theta(W)
    W *=expOp
    W = X_Theta__To__X_P(W)
    return W

In [ ]:
def Kinetic(p):
    return 0.5*(1./mass)*p**2

def Potential(x):
#        return 0.5*k*x**2  
    return 0.05*k*x**4

def createSepPropagatorExponentials_Separable(dt):
    ''' Exponential Propagators for the Separable Case'''
    global potentialPropagatorFactor, kineticPropagatorFactor
    potentialPropagatorFactor  = MoyalGenerator_G(dt, Potential, halveStep, X, Theta,   1, 'Separable')
    kineticPropagatorFactor    = MoyalGenerator_G(dt, Kinetic,   1,         P, -Lambda, 0, 'Separable')    

def evolStep_SeparableHamiltonian_Exp(dt, W):
    unusedSlot = 0
    W = PotentialStep(W,potentialPropagatorFactor,unusedSlot)                
    W = KineticStep(W,kineticPropagatorFactor,unusedSlot)
    if StrangSplitting:
        W = PotentialStep(W,potentialPropagatorFactor,unusedSlot)                
    return W

In [ ]:
def createPropagatorExponentials_KERR(dt):
    global expv0V, expv1V, expv2V, expt1T, expt2T, expdtVx4half, expdtVx4, expdtTp4, v1, v2, v0, t1, t2 

    def Vx4(xx):
        return xx**4/4 - (1-keep_if_9Exp_case) * CorrectionPotential_7Exp(xx)

    def Tp4(pp):
        return pp**4/4

    def Veff(xx):
        '''Effective Potentials for x^2 p^2 Propagator'''
        return xx**4/12

    def Teff(pp):
        '''Effective Potentials for x^2 p^2 Propagator'''
        return pp**2/2/np.sqrt(2)

    # Defining Chin's parameters DOI: 10.1103/PhysRevE.80.037701
    t2Magn = np.cbrt(6)
    v1 =  1/t2Magn**2
    v2 = -v1/2 * keep_if_9Exp_case
    v0 = -2*(v1+v2)
    t1 =  t2Magn
    t2 = -t2Magn

    expv0V        = MoyalGenerator_G(dt, Veff, v0, X, Theta, 1, 'ChinsCase')
    expv1V        = MoyalGenerator_G(dt, Veff, v1, X, Theta, 1, 'ChinsCase')
    expv2V        = MoyalGenerator_G(dt, Veff, v2, X, Theta, 1, 'ChinsCase')

    expt1T        = MoyalGenerator_G(dt, Teff, t1, P, -Lambda, 0, 'ChinsCase')    
    expt2T        = MoyalGenerator_G(dt, Teff, t2, P, -Lambda, 0, 'ChinsCase')

    # These three are all yielding the Default 'Separable' Case
    expdtVx4half = MoyalGenerator_G(dt, Vx4, halveStep, X, Theta,   1)
    expdtVx4     = MoyalGenerator_G(dt, Vx4, 1,         X, Theta,   1, )
    expdtTp4     = MoyalGenerator_G(dt, Tp4, 1,         P, -Lambda, 0, 'Separable')  

In [ ]:
def evolStepKerr1_9_Exp(dt, W, unused_dummy):
    '''Chin 9-Exponentials method'''

    if StrangSplitting == True:
        W = PotentialStep( W, expdtVx4half,  unused_dummy)
    else:
        W = PotentialStep( W, expdtVx4,  unused_dummy)

    W = PotentialStep( W, expv2V,   unused_dummy)
    W = KineticStep(   W, expt2T,   unused_dummy)
    W = PotentialStep( W, expv1V,   unused_dummy)
    W = KineticStep(   W, expt1T,   unused_dummy)
    W = PotentialStep( W, expv0V,   unused_dummy)
    W = KineticStep(   W, expt1T,   unused_dummy)
    W = PotentialStep( W, expv1V,   unused_dummy)
    W = KineticStep(   W, expt2T,   unused_dummy)
    W = PotentialStep( W, expv2V,   unused_dummy)

    W = KineticStep(   W, expdtTp4, unused_dummy)

    if StrangSplitting == True:
        W = PotentialStep( W, expdtVx4half,  unused_dummy)

    W = np.real(W)

    return W

In [ ]:
def evolStepKerr1_7_Exp(dt, W, unused_dummy):
    '''Chin 7-Exponentials method'''

    if StrangSplitting == True:
        W = PotentialStep( W, expdtVx4half,  unused_dummy)
    else:
        W = PotentialStep( W, expdtVx4,  unused_dummy)

    W = KineticStep(   W, expt2T,   unused_dummy)
    W = PotentialStep( W, expv1V,   unused_dummy)
    W = KineticStep(   W, expt1T,   unused_dummy)
    W = PotentialStep( W, expv0V,   unused_dummy)
    W = KineticStep(   W, expt1T,   unused_dummy)
    W = PotentialStep( W, expv1V,   unused_dummy)
    W = KineticStep(   W, expt2T,   unused_dummy)
    
    W = KineticStep(   W, expdtTp4, unused_dummy)

    if StrangSplitting == True:
        W = PotentialStep( W, expdtVx4half,  unused_dummy)

    W = np.real(W)

    return W

In [ ]:
Energy_Landscape = np.array(Hamiltonian(X,P))

In [ ]:
def IntegrateXP(F):
    ''' Phase Space integration routine '''
    return integrate.simps(integrate.simps(F.real, pvector), xvector)

In [ ]:
def Plot_Wigner(t, dt, plots_directory, W):
    ''' Plot Wigner Distribution '''
    zero=0 # I am meant to pass only the V(xvector) instead of Hamiltonian(xvector,zero*pvector)

    WignerPlot_221104.PlotWignerDistr_Projections_Title(np.transpose(W.real),t*dt,xvector,pvector,1.8,1.8,\
        True,(r"$W_{[\Psi_0(%6.3f,%6.3f)]}(t=%6.3f)$" % (x_off,p_off,t*dt)),Hamiltonian(xvector,zero*pvector),0,0,0,1,0,KerrCase)
    
    filename = f"t_{t*dt:3.3f}_[{discretizationX}x{discretizationP}]_R_{R}_{motto}_{'_2ndStrang' if StrangSplitting else ''}_psi({x_off:.3f},{p_off:.3f})_dt{dt}"
    plt.savefig(f'{plots_directory}/{t}__{filename}.png')#df')
    plt.close('all')

In [ ]:
def span(X, X0):
    return max((np.max(X) - X0), (X0 - np.min(X)))

## Comparison plots of fluctuation data in frames

In [ ]:
def plot_energy_scaling(energy_data, suptitleString):
    ''' Plot energy(T) scaling '''
    fig,axs = plt.subplots(3,3,figsize=(19.2,14.4))
    fig.suptitle(f"Scaling_ΔE_{suptitleString}")#EnergyScaling}")

    for idx,(x_off,p_off) in enumerate(initial_offsets):
        i,j = subplot_indices[idx]
        # Get data
        logDts = []
        logMaxErrors = []
        for dt,timeStepsN,Es in energy_data[i][j]:
            logDts.append(np.log10(dt))
            logMaxErrors.append(np.log10(max(Es)-min(Es)))
#            logMaxErrors.append(np.log10(np.abs(span(Es, Es[0]))))
        axs[i, j].scatter(logDts, logMaxErrors, label="Data Points", color='blue')
        # Best fit
        gradient, intercept, r_value, p_value, std_err = stats.linregress(logDts, logMaxErrors)
        axs[i, j].plot(logDts, [gradient*logdt+intercept for logdt in logDts], label="Best-Fit Line", color='red')
        axs[i, j].legend()
        absE = 10.0**(-intercept)
        
        plt.setp(axs[i, j], xlabel=r"$\log_{10} \left (dt \right )$", ylabel=r"$\log_{10} | E_{max} - E_{min} |$", \
                 title=f"E(Δt)[$\Psi_{0}$({x_off:.3f},{p_off:.3f})]={10.0**(-intercept):.3E} x 10^(-{gradient:.3f}Δt)")
        print(f"Energy-Gradient[psi({x_off:.3f},{p_off:.3f})]={gradient}")
    axs[1, 1].axis("off")
    plt.savefig(f"{discretizationX}x{discretizationP}_Energy_Scaling_R_{R}_{motto}{'_Kerr' if KerrCase else ''}{'_2nd_Strang' if StrangSplitting else ''}_dt{list_dt_4_Kerr if KerrCase else list_dt_4_separableSystem}.png")
    plt.savefig(f"{discretizationX}x{discretizationP}_Energy_Scaling_R_{R}_{motto}{'_Kerr' if KerrCase else ''}{'_2nd_Strang' if StrangSplitting else ''}_dt{list_dt_4_Kerr if KerrCase else list_dt_4_separableSystem}.pdf")

In [ ]:
def plot_energy_fluctuations(energy_data, suptitleString):
    ''' Plot energy fluctuations(t) '''

    fig,axs = plt.subplots(3,3,figsize=(19.2,14.4))
    fig.suptitle(f"E(t)_{suptitleString}")
    
    for idx,(x_off,p_off) in enumerate(initial_offsets):
        i,j = subplot_indices[idx]
        print(f"Energy-Fluctuations[psi({x_off:.3f},{p_off:.3f})]")	
        for dt,timeStepsN,Es in energy_data[i][j]:
            axs[i, j].plot(np.linspace(0, dt*timeStepsN, num=timeStepsN), Es, label=f"dt={dt}")
        axs[i, j].legend()
        plt.setp(axs[i, j], xlabel='t', ylabel='E', title=f'psi({x_off:.3f},{p_off:.3f})')
    axs[1, 1].axis('off')
    plt.savefig(f"{discretizationX}x{discretizationP}_Energy_Fluctuations_R_{R}_{motto}{'_Kerr' if KerrCase else ''}{'_2ndStrang' if StrangSplitting else ''}_dt{list_dt_4_Kerr if KerrCase else list_dt_4_separableSystem}.png")

In [ ]:
def plot_Norm_scaling(norm_data, ylabelString, suptitleString):
    ''' Plot scaling Norm(T) '''

    fig,axs = plt.subplots(3,3,figsize=(19.2,14.4))
    fig.suptitle(f"Norm(T):_{ylabelString}_{suptitleString}")
    for idx,(x_off,p_off) in enumerate(initial_offsets):
        i,j = subplot_indices[idx]
        logDts = []
        logMaxErrors = []
        for dt,timeStepsN,norm in norm_data[i][j]:
            logDts.append(np.log10(dt))
            logMaxErrors.append(np.log10(np.abs(norm[timeStepsN-1])))
        axs[i, j].scatter(logDts, logMaxErrors, linewidth=3.0, label="Data Points", color='black')
        gradient, intercept, r_value, p_value, std_err = stats.linregress(logDts, logMaxErrors)
        axs[i, j].plot(logDts, [gradient*logdt+intercept for logdt in logDts], label="Best-Fit Line", color='green')
        axs[i, j].legend()
        
        plt.setp(axs[i, j], \
                 xlabel=r"$\log_{10} \left (dt \right )$", \
                 ylabel=r"$\log_{10}$"f"{ylabelString}", \
                 title=f"{ylabelString}(Δt)[$\Psi_{0}$({x_off:.1f},{p_off:.1f})]={10.0**(-intercept):.2E} x 10^(-{gradient:.2f}Δt)"
                )
        print(r"{ylabelString}-Scaling Gradient[psi({x_off:.3f},{p_off:.3f})]={gradient}")
    axs[1, 1].axis("off")
    plt.savefig(f"{discretizationX}x{discretizationP}_Scaling_{ylabelString}_R_{R}_{motto}{'_Kerr' if KerrCase else ''}{'_2ndStrang' if StrangSplitting else ''}_dt{list_dt_4_Kerr if KerrCase else list_dt_4_separableSystem}.png")
    plt.savefig(f"{discretizationX}x{discretizationP}_Scaling_{ylabelString}_R_{R}_{motto}{'_Kerr' if KerrCase else ''}{'_2ndStrang' if StrangSplitting else ''}_dt{list_dt_4_Kerr if KerrCase else list_dt_4_separableSystem}.pdf")

In [ ]:
def plot_overlap_fluctuations(norm_data, ylabelString, suptitleString):
    ''' Plot Norm(t) '''
    fig,axs = plt.subplots(3,3,figsize=(19.2,14.4))
    fig.suptitle(f"Norm(t):_{ylabelString}_{suptitleString}")
        
    for idx,(x_off,p_off) in enumerate(initial_offsets):
        i,j = subplot_indices[idx]
        for dt,timeStepsN,norm in norm_data[i][j]:
            axs[i, j].plot(np.linspace(0, dt*timeStepsN, num=timeStepsN), norm, label=f"dt={dt}")
        axs[i, j].legend()
        plt.setp(axs[i, j], xlabel='t', ylabel=f'{ylabelString}', title=f'psi({x_off:.3f},{p_off:.3f})')
    axs[1, 1].axis('off')
    plt.savefig(f"{discretizationX}x{discretizationP}_Fluctuations_{ylabelString}_R_{R}_{motto}{'_2ndStrang' if StrangSplitting else ''}_dt{list_dt_4_Kerr if KerrCase else list_dt_4_separableSystem}.png")

In [ ]:
def plots_dir(dt):
    ''' declaration of save path'''
    return f"wigframes_[{discretizationX}x{discretizationP}]_R_{R}_{motto}_dt{list_dt_4_Kerr  if KerrCase else list_dt_4_separableSystem}{'_2ndStrang' if StrangSplitting else ''}/psi({x_off:.3f},{p_off:.3f})/{dt}/"

In [ ]:
def normOverlap(W1, W2):
    ''' 0 =< ... =< 1 '''
    return 1 - 2*np.pi*IntegrateXP(W1*W2)

def normL2(W1, W2):
    ''' 0 =< ... =< 1 '''
    return np.sqrt(np.pi*IntegrateXP((W1-W2)**2))

def normLoo(W1, W2):
    ''' 0 =< ... =< 1/pi '''
    return np.max(np.abs(W1-W2))

# Concentric Loops for FRAMES & TIMES

In [ ]:
## Beginning of FRAMES  Loop
for idx,(x_off,p_off) in enumerate(initial_offsets):
    i,j = subplot_indices[idx]
    Winit = (1./np.pi)*np.exp(-(X-x_off)**2-(P-p_off)**2) + 0j
    
    if only_run_until_HalfTime and KerrCase:
        W_target =  (1./2/np.pi)*np.exp(-(X+p_off)**2-(P-x_off)**2) \
                 +  (1./2/np.pi)*np.exp(-(X-p_off)**2-(P+x_off)**2) \
                 -  (1./1/np.pi)*np.exp(-(X)**2-(P)**2) * np.sin(2*(x_off*X+p_off*P))
    else:
        W_target = copy.deepcopy(Winit)
    
    ## Begin of the dt- LOOP which varies the time-steps for Scaling Information
    for dt in (list_dt_4_separableSystem if not KerrCase else list_dt_4_Kerr):
        os.makedirs(plots_dir(dt), exist_ok=True)

        # number of steps between output for intermediate plots 
        timeStepsN = round(period/dt) + 1
                
        # Defining the propagator factors as functions of  'dt'
        if KerrCase:
            createPropagatorExponentials_KERR(dt)                   
        else:
            createSepPropagatorExponentials_Separable(dt)
            
        W = intialize_EnergySeries_OverlapSeries__State_and_Plot_State(dt, plots_dir(dt), Winit, W_target) 
        
        Plot_Wigner(0, 100000000, plots_dir(dt), W_target)

        ## Time  Loop
        for t in range(1,timeStepsN):                
            if KerrCase:
                if exp7:
                    W = evolStepKerr1_7_Exp(dt, W, 0)
                else:
                    W = evolStepKerr1_9_Exp(dt, W, 0)
            else: # separable Hamiltonian
                W = evolStep_SeparableHamiltonian_Exp(dt, W)
                
            E = IntegrateXP(Energy_Landscape*W)
            Es.append(E)
            
            WoW.append( normOverlap(W, W_target))     
            WL2.append( normL2(     W, W_target))
            WLoo.append(normLoo(    W, W_target))    
            
            if (t % int(timeStepsN/29) == 1):
                print(f"dt={dt} T_run:{time.time()-start_time:.1f} sec [{t}/{timeStepsN}:t={dt*t:.4f}] {motto} R={R}:psi_0({x_off:.3f},{p_off:.3f}), Grid[{discretizationX}x{discretizationP}]   E[{t}]={E:.3f}  <Wo|W>[{t}]={WoW[-1]:.3f}")

            # Intermediate plots
#            if (t % int(timeStepsN/3 == 1):
            if (t % int(timeStepsN/(3+1)) == 1):
                Plot_Wigner(t, dt, plots_dir(dt), W)


        # Plot final state
        Plot_Wigner(t, dt, plots_dir(dt), W)
        # Store data for current frame
        energy_data[i][j].append((dt,timeStepsN,Es.copy()))
        WoW_data[i][j].append((dt,timeStepsN,WoW.copy()))        
        DWL2_data[i][j].append((dt,timeStepsN,WL2.copy()))
        DWLoo_data[i][j].append((dt,timeStepsN,WLoo.copy()))

        ## END of Time  Loop
    ## END of the dt- LOOP which varies the time-steps for Scaling Information
## END of FRAMES  Loop

In [ ]:
#
## END of MAIN Program: write results to image files
# 
suptitleString=f"{motto}_Grid[{discretizationX}x{discretizationP}]_R={R}_dt={list_dt_4_Kerr if KerrCase else list_dt_4_separableSystem}_Order[{'2Strang' if StrangSplitting else '1?'}]"
compTime = f"[{time.time()-start_time:.1f} sec]"
print("Total Calculation Time:",compTime)
suptitleString = f"{suptitleString}_(RunT[{compTime:s}])"
plot_energy_scaling(energy_data, suptitleString)
plot_energy_fluctuations(energy_data, suptitleString)

plot_overlap_fluctuations(WoW_data, r"$1-<W(t)|Wo>$", f"{suptitleString}")
plot_overlap_fluctuations(DWL2_data, r"$<|W(t)-Wo|^{2}>^{0.5}$", f"{suptitleString}")
plot_overlap_fluctuations(DWLoo_data, r"$||W(t)-Wo||_{oo}$", f"{suptitleString}")

plot_Norm_scaling(WoW_data, r"$(1-<W(T)|Wo>)$", f"{suptitleString}")
plot_Norm_scaling(DWL2_data, r"$<|W(T)-Wo|^{2}>^{0.5}$", f"{suptitleString}")
plot_Norm_scaling(DWLoo_data, r"$||W(T)-Wo||_{oo}$", f"{suptitleString}")

compTime = f"[{time.time()-start_time:.1f} sec]"
print("Total Calculation Time:",compTime)

In [ ]:
def intialize_EnergySeries_OverlapSeries__State_and_Plot_State(dt, plots_directory, Winit, W_target):
    global Es, WoW, WL2, WLoo

    Plot_Wigner(0, dt, plots_directory, Winit)
    W    = np.array(Winit)
    Es   = [IntegrateXP(Energy_Landscape*Winit)]
    WoW  = [normOverlap(Winit, W_target)]
    WL2  = [normL2(Winit, W_target)]
    WLoo = [normLoo(Winit, W_target)]
    return W